Tool updated: 2024-06-03, Henrik Loecke

Tool summary:
This tool creates RAWN sheets, based on the MIKE+ model and MPF population file. It traces through the model 


In [1]:
#Permanent cell 1
import arcpy
import pandas as pd
import sqlite3
import math
import numpy as np
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import ctypes
import traceback
import shutil
MessageBox = ctypes.windll.user32.MessageBoxA

In [2]:
#Permanent cell 2
def sql_to_df(sql,model):
    con = sqlite3.connect(model)
    df = pd.read_sql(sql, con)
    con.close()
    return df

def execute_sql(sqls,model):
    con = sqlite3.connect(model)
    cur = con.cursor()
    if type(sqls) == list:
        for sql in sqls:
            cur.execute(sql)
    else:         
        cur.execute(sqls)
    cur.close()
    con.commit()
    con.close()

In [119]:
#Permanent cell 3
# User Input, to move to separate sheet so no permanent cell
#stop trace if more pipes than max_steps traced from catchment, must be an endless loop. 

model = 'FSA'

max_steps = 1000 
use_formula = True

check_scenario = True #Scenario check is for max active altid. In some hypothetical setups it can be incorrect. 
#                      If you verify the scenario is correct (open model to check) but get the error, set this check to False
#                      A more robust check may be developed in the future.
update_field_in_model = True
update_field = 'Description'
global_output_folder = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\Rawn_Tool\Output"

if model == 'NSSA':   
    model_output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\04_ANALYSIS_WORK\RAWN_From_Model"
    model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\Automation\NSSA_Base_2018pop.sqlite"
    pop_book = r"\\prdsynfile01\LWS_Modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\MPF4_Temp_Hold\NSSA_Master_Population_File_4_No_2237_ResArea.xlsx"
    pop_sheet = 'MPF Update 4'
    scenario = 'Base'
    line_exclusions = []
    years = [2060,2070,2080,2090,2100]
    
if model == 'FSA':   
    model_output_folder = r'\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\RAWN_From_Model'
    model_path = r"J:\SEWER_AREA_MODELS\FSA\04_ANALYSIS_WORK\Model_Result_To_GIS\FSA_Base_2021pop.sqlite"
    pop_book = r"J:\SEWER_AREA_MODELS\FSA\02_MODEL_COMPONENTS\04_DATA\01. POPULATION\FSA_Master_Population_File.xlsx"
    pop_sheet = 'MPF Update 16'
    scenario = '2030_Network'
    line_exclusions = ['GoldenEar_Dummy_Pump','GoldenEarSSOLink','GoldenEar_SSO_Tank_Cell_Dummy_Link3','MH35_Orifice','44473']
    years = [2060,2070,2076]
    
sewer_area = model
gdb_name = 'RAWN.gdb'
gdb_name_dissolve = 'RAWN_Dissolve.gdb' #To keep clutter out of main database

#Options to skip time consuming steps during debug, must be True during production runs
run_dissolve = True
run_jpg = True
run_import = True
run_html = False


In [42]:
#Permanent cell 4
#Set up column names

try:
    
    categories = ['res','com','ind','inst','infl','infi']

    mpf_col_dict = {}

    area_col_dict = {}
    area_col_dict['res'] = 'Area_Res'
    area_col_dict['com'] = 'Area_Com'
    area_col_dict['ind'] = 'Area_Ind'
    area_col_dict['inst'] = 'Area_Inst'
    area_col_dict['ini'] = 'Area_Total'

    per_unit_dict = {}
    per_unit_dict['res'] = 320
    per_unit_dict['com'] = 33700 
    per_unit_dict['ind'] = 56200
    per_unit_dict['inst'] = 33700
    per_unit_dict['infl'] = 5600
    per_unit_dict['infi'] = 5600

    unit_dict = {}
    unit_dict['res'] = 'L/c/d'
    unit_dict['com'] = 'L/ha/d'
    unit_dict['ind'] = 'L/ha/d'
    unit_dict['inst'] = 'L/ha/d'
    unit_dict['infl'] = 'L/ha/d'


    header_dict = {}
    # header_dict['gen'] = ['GENERAL INFO',['TYPE','MODELID','CATCHMENT','ID','YEAR','LOCATION']]
    header_dict['gen'] = ['GENERAL INFO',['TYPE','CATCHMENT','YEAR','LOCATION']]
    header_dict['res'] = ['RESIDENTIAL',['AREA (Ha)','POPULATION','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['com'] = ['COMMERCIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ind'] = ['INDUSTRIAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['inst'] = ['INSTITUTIONAL',['AREA (Ha)','AVG. FLOW (L/s)','PEAK FLOW (L/s)']]
    header_dict['ini'] = ['INFLOW / INFILTRATION',['AREA (Ha)','INFLOW (L/s)','INFILTRATION (L/s)']]
    header_dict['flow'] = ['FLOWS',['AVG. SAN. FLOW (L/s)','ADWF (L/s)','PWWF (L/s)']]

    avg_calc_dict = {}
    #Items: [Keyword (upper Excel header),Type ('lower Excel header'),Average(lower Excel header),Unit flow cell address, quantifyer column]
    avg_calc_dict['res'] = ['RESIDENTIAL','POPULATION','AVG. FLOW (L/s)','$D$3','H']
    avg_calc_dict['com'] = ['COMMERCIAL','AREA (Ha)','AVG. FLOW (L/s)','$D$4','K']
    avg_calc_dict['ind'] = ['INDUSTRIAL','AREA (Ha)','AVG. FLOW (L/s)','$D$5','N']
    avg_calc_dict['inst'] = ['INSTITUTIONAL','AREA (Ha)','AVG. FLOW (L/s)','$D$6','Q']
    avg_calc_dict['infl'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFLOW (L/s)','$D$7','T']
    avg_calc_dict['infi'] = ['INFLOW / INFILTRATION','AREA (Ha)','INFILTRATION (L/s)','$D$7','T']

    header_tuples = []
    for header in header_dict:
        for sub_header in (header_dict[header][1]):
            header_tuples.append((header_dict[header][0],sub_header))
    header_tuples

    # columns_multiindex = pd.MultiIndex.from_tuples(header_tuples,names=['Category', 'Subcategory'])
    columns_multiindex = pd.MultiIndex.from_tuples(header_tuples)
    df_template = pd.DataFrame(columns=columns_multiindex)

    info_list = []
    for item in unit_dict:
        info_list.append([avg_calc_dict[item][0],per_unit_dict[item],unit_dict[item]])
    info_df = pd.DataFrame(info_list,columns=['DESCRIPTION','AVG. FLOW','UNITS'])
    info_df.set_index('DESCRIPTION',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 4', b'Error', 0)
    raise ValueError("Error")

In [29]:
#Permanent cell 5
#Import population
try:
    pop_df = pd.read_excel(pop_book,sheet_name=pop_sheet,dtype={'Catchment': str})#[['Catchment','Year','Pop_Total']]
    pop_df.rename(columns={"Pop_Total": "Population"},inplace=True)
    pop_df = pop_df[['Catchment','Year','Pop_ResLD','Pop_ResHD','Pop_Mixed','Population','Area_ResLD','Area_ResHD','Area_Mixed','Area_Com','Area_Ind','Area_Inst']]
    pop_df.fillna(0, inplace=True) #Fill NA with 0 or the sum of all will be NA
    pop_df['Area_Res'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed
    pop_df['Area_Total'] = pop_df.Area_ResLD + pop_df.Area_ResHD + pop_df.Area_Mixed + pop_df.Area_Com + pop_df.Area_Ind + pop_df.Area_Inst
    pop_df['Population_Sum_Check'] = pop_df.Pop_ResLD + pop_df.Pop_ResHD + pop_df.Pop_Mixed
        
    pop_sum_total_col = int(pop_df.Population.sum())
    pop_sum_sub_cols = int(pop_df.Pop_ResLD.sum() + pop_df.Pop_ResHD.sum() + pop_df.Pop_Mixed.sum())
    pop_df['Key'] = sewer_area + '@' + pop_df.Catchment + '@' + pop_df['Year'].astype(str)
    pop_df.set_index('Key',inplace=True)

    if pop_sum_total_col != pop_sum_sub_cols:
          print("Warning. The sum of 'Population' (" + str(pop_sum_total_col) + ") is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (" + str(pop_sum_sub_cols) + ")") 

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 5', b'Error', 0)
    raise ValueError("Error")

Warning. The sum of 'Population' (113071710) is different than the sum of 'Pop_ResLD' + 'Pop_ResHD' + 'Pop_Mixed' (112676302)


In [114]:
#Permanent cell 6
#Import model data

try:
    node_types = {}
    node_types[1] = 'Manhole'
    node_types[2] = 'Basin'
    node_types[3] = 'Outlet'
    node_types[4] = 'Junction'
    node_types[5] = 'Soakaway'
    node_types[6] = 'River Junction'

    sql = "SELECT max(AltID) FROM msm_Loadpoint WHERE Active = 1"
    altid = sql_to_df(sql,model_path).iloc[0,0]

    sql = "SELECT max(AltID) FROM msm_Link WHERE Active = 1"
    altid = sql_to_df(sql,model_path).iloc[0,0]

    if altid == 0:
        active_scenario = 'Base'
    else:
        sql = "SELECT MUID FROM m_ScenarioManagementAlternative WHERE GroupID = 'CS_Network' AND AltID = " + str(altid)
        active_scenario = sql_to_df(sql,model_path).iloc[0,0]

    if scenario != active_scenario:
        raise ValueError(f'Scenario {scenario} was requested in the user input but scenario {active_scenario} is active in the model')
    
    sql = "SELECT catchid AS Catchment, nodeid AS Connected_Node FROM msm_Catchcon WHERE Active = 1"
    catchments = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], uplevel AS Outlet_Level FROM msm_Link WHERE Active = 1"
    lines = sql_to_df(sql,model_path)

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Orifice WHERE Active = 1"
    orifices = sql_to_df(sql,model_path)
    lines = pd.concat([lines,orifices])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], invertlevel AS Outlet_Level FROM msm_Valve WHERE Active = 1"
    valves = sql_to_df(sql,model_path)
    lines = pd.concat([lines,valves])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], crestlevel AS Outlet_Level FROM msm_Weir WHERE Active = 1"
    weirs = sql_to_df(sql,model_path)
    lines = pd.concat([lines,weirs])

    sql = "SELECT muid AS MUID, fromnodeid AS [From], tonodeid as [To], startlevel AS Outlet_Level FROM msm_Pump WHERE Active = 1"
    pumps = sql_to_df(sql,model_path)
    lines = pd.concat([lines,pumps])

    lines['Outlet_Level'].fillna(-9999, inplace=True)
    
    lines = lines[~lines['MUID'].isin(line_exclusions)]

    sql = "SELECT muid, acronym, assetname FROM msm_Node WHERE active = 1"
    node_id_df = sql_to_df(sql,model_path)
    node_id_df = node_id_df[(node_id_df.assetname.str[:2]=='MH') & (node_id_df.assetname.str.len() > 2) & (node_id_df.acronym.notna())]
    node_id_df.rename(columns={'muid':'Node'},inplace=True)
    node_id_df['ID'] = node_id_df.acronym + '_' + node_id_df.assetname
    node_id_df = node_id_df[['Node','ID']]
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 6', b'Error', 0)
    raise ValueError("Error")

In [115]:
#Permanent cell 7
#Trace the model

try:
    accumulated_catchment_set = set()
    accumulated_node_set = set()

    for index1, row1 in catchments.iterrows():
        catchment = row1['Catchment']
        nodes = [row1['Connected_Node']]
        start_node = row1['Connected_Node']
        steps = 0

        accumulated_catchment_set.add((start_node,catchment))

        while steps <= max_steps:
            steps += 1
            downstream_df = lines[lines['From'].isin(nodes)]  

            if len(downstream_df) > 0:
                nodes = list(downstream_df.To.unique())

                nodes = [node for node in nodes if len(node)>0]
                for node in nodes:
                    accumulated_catchment_set.add((node,catchment))       
            else:
                break
            if steps == max_steps:
                raise ValueError("Maximum steps were reached, indicating a loop. Last node traced is '" + node + "'")

            accumulated_catchment_set.add((node,catchment))

    accumulation_df = pd.DataFrame(accumulated_catchment_set,columns=['Node','Catchment'])
    accumulation_df = pd.merge(accumulation_df,node_id_df,how='inner',on=['Node'])
    data = {
        ('GENERAL INFO', 'CATCHMENT'): accumulation_df.Catchment,
        ('GENERAL INFO', 'NODE'): accumulation_df.Node,
        ('GENERAL INFO', 'ID'): accumulation_df.ID,
    }

    # Create a DataFrame with MultiIndex columns
    accumulation_df = pd.DataFrame(data)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 7', b'Error', 0)
    raise ValueError("Error")



In [126]:
max([4,3])

4

In [129]:
(1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5))

0       2.327269
1       2.292673
2       2.273400
3       2.279883
4       2.249357
          ...   
5260    2.776005
5261    2.767155
5262    2.261884
5263    2.232192
5264    2.215520
Name: (RESIDENTIAL, POPULATION), Length: 5265, dtype: float64

In [130]:
max([(1 + 14 / (4 + (55/ 1000) ** 0.5)),1.5])


4.3061592328707174

In [138]:
#Permanent cell 8
#Calculate RAWN

try:
    catchments = list(pop_df.Catchment.unique())

    catchment_df = df_template.copy()
    for catchment in catchments:
        for year in years:
            key = model + '@' + catchment + '@' + str(year)
            catchment_df.loc[key,('GENERAL INFO','TYPE')] = 'Manhole'
            catchment_df.loc[key,('GENERAL INFO','CATCHMENT')] = catchment
            catchment_df.loc[key,('GENERAL INFO','YEAR')] = year
            catchment_df.loc[key,('GENERAL INFO','LOCATION')] = model
            for area_col_dict_key in area_col_dict:
                catchment_df.loc[key,(header_dict[area_col_dict_key][0],'AREA (Ha)')] = pop_df.loc[key,area_col_dict[area_col_dict_key]]
            catchment_df.loc[key,('RESIDENTIAL','POPULATION')] = pop_df.loc[key,'Population']
            san_flow = 0
            adwf = 0
            for avg_calc_dict_key in avg_calc_dict:
                input1 = catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][1])]
                input2 = per_unit_dict[avg_calc_dict_key]
                avg_flow = input1 * input2 / 86400
                if avg_calc_dict_key not in ['infl','infi']:
                    san_flow += avg_flow
                if avg_calc_dict_key not in ['infl']:
                    adwf += avg_flow    
                catchment_df.loc[key,(avg_calc_dict[avg_calc_dict_key][0],avg_calc_dict[avg_calc_dict_key][2])] = avg_flow
            catchment_df.loc[key,('FLOWS','AVG. SAN. FLOW (L/s)')] = san_flow
            catchment_df.loc[key,('FLOWS','ADWF (L/s)')] = adwf


    catchment_node_df = accumulation_df.merge(catchment_df,on=[('GENERAL INFO','CATCHMENT')],how='inner')
    node_df = catchment_node_df.copy()
    node_df.drop(columns=[('GENERAL INFO','CATCHMENT')],inplace=True)
    node_df = node_df.groupby([('GENERAL INFO','NODE'),('GENERAL INFO','TYPE'),('GENERAL INFO','YEAR'),('GENERAL INFO','LOCATION'),('GENERAL INFO','ID')]).sum()
    node_df.reset_index(inplace=True)
    node_df[('RESIDENTIAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (node_df[('RESIDENTIAL','POPULATION')] / 1000) ** 0.5)),1.5) * node_df[('RESIDENTIAL','AVG. FLOW (L/s)')]
    node_df[('COMMERCIAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (per_unit_dict['com'] * node_df[('COMMERCIAL','AREA (Ha)')]/(per_unit_dict['res'] * 1000)) ** 0.5))*0.8,1.5)*node_df[('COMMERCIAL','AVG. FLOW (L/s)')]
    node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = np.maximum((1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)),1.5) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

#     node_df[('INSTITUTIONAL','PEAK FLOW (L/s)')] = (1 + 14 / (4 + (per_unit_dict['inst'] * node_df[('INSTITUTIONAL','AREA (Ha)')] / (per_unit_dict['res'] * 1000)) ** 0.5)) * node_df[('INSTITUTIONAL','AVG. FLOW (L/s)')]

    
    mask = node_df[('INDUSTRIAL', 'AREA (Ha)')] != 0 #Avoid error from log(0)
    node_df.loc[mask, ('INDUSTRIAL', 'PEAK FLOW (L/s)')] = np.maximum(
        0.8 * (
            1 + 14 / (
                4 + (node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] * per_unit_dict['ind'] / (per_unit_dict['res'] * 1000)) ** 0.5
            )
        ) * np.where(
            node_df[('INDUSTRIAL', 'AREA (Ha)')][mask] < 121,
            1.7,
            2.505 - 0.1673 * np.log(node_df[('INDUSTRIAL', 'AREA (Ha)')][mask])
        ), 
        1.5
    ) * node_df[('INDUSTRIAL', 'AVG. FLOW (L/s)')][mask]
    

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 8', b'Error', 0)
    raise ValueError("Error")



In [9]:
#Permanent cell 9
#Import GIS from the model

try:
    if run_import:

        out_path = global_output_folder + '\\' + gdb_name

        if not os.path.isdir(out_path):
            arcpy.management.CreateFileGDB(global_output_folder, gdb_name)

        arcpy.env.workspace = out_path
        sr = arcpy.SpatialReference(26910)

        layers = ['msm_CatchCon','msm_Catchment','msm_Link','msm_Node','msm_Pump','msm_Weir','msm_Orifice','msm_Valve']

        for layer in layers:

            arcpy.management.MakeFeatureLayer(model_path + '\\' + layer, "temp_layer", "Active = 1")

            if arcpy.Exists(out_path + '\\' + layer):
                print('Appending ' + layer)
                arcpy.management.DeleteFeatures(layer)
                arcpy.management.Append("temp_layer", layer, "NO_TEST")
            else:  
                print('Creating ' + layer)

                arcpy.conversion.FeatureClassToFeatureClass("temp_layer", out_path, layer + '_Test')
                if layer == 'msm_Catchment':
                    arcpy.management.AddField('msm_catchment', "Drains_To", "TEXT")
                arcpy.DefineProjection_management(layer, sr)

            arcpy.management.Delete("temp_layer")
            
            
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 9', b'Error', 0)
    raise ValueError("Error")


Appending msm_CatchCon
Appending msm_Catchment
Appending msm_Link
Appending msm_Node
Appending msm_Pump
Appending msm_Weir
Appending msm_Orifice
Appending msm_Valve


In [10]:
#Permanent cell 10
#Create merge_df,minimizing the number of computation heavy dissolves that need to be done.

try:
    merge_set = set()

    rank_df = accumulation_df[[('GENERAL INFO','NODE'),('GENERAL INFO','CATCHMENT')]].groupby([('GENERAL INFO','NODE')]).count()

    # rank_df.reset_index(inplace=True)
    rank_df.columns = ['Catchment_Count']
    max_catchments = max(rank_df.Catchment_Count)
    rank_df.sort_values(by=['Catchment_Count'],inplace=True)
    # rank_df.reset_index(inplace=True)

    catchment_list = []
    merge_set = set()
    for index, row in rank_df.iterrows():

        catchments = list(accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==index][('GENERAL INFO','CATCHMENT')].unique())
        catchments = tuple(sorted(catchments))
        catchment_list.append(catchments)
        merge_set.add(catchments)


    rank_df['Catchments'] = catchment_list
    rank_df['Node'] = rank_df.index


    merge_list = []
    for i, catchments in enumerate(merge_set):
        merge_id = 'Merge_ID_' + str(i)
        merge_list.append([merge_id,catchments])

    merge_df = pd.DataFrame(merge_list,columns=['Merge_ID','Catchments'])
    merge_df['Catchment_Count'] = merge_df['Catchments'].apply(len)
    merge_df.sort_values(by=['Catchment_Count'],ascending=False,inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    simpler_merge = []
    for index1, row1 in merge_df.iterrows():
        catchments1 = list(row1['Catchments'])
        for index2, row2 in merge_df[index1+1:].iterrows():
            catchments2 = row2['Catchments']

            if len(catchments1) >= len(catchments2):
                if all(item in catchments1 for item in catchments2):
                    catchments1 = [catchment for catchment in catchments1 if catchment not in catchments2]
                    catchments1.append(row2['Merge_ID'])
        simpler_merge.append(catchments1)

    merge_df['To_Dissolve'] = simpler_merge
    merge_df.sort_values(by=['Catchment_Count'],inplace=True)
    merge_df.reset_index(inplace=True,drop=True)

    rank_df = pd.merge(rank_df,merge_df[['Merge_ID','Catchments']], on=['Catchments'],how='inner')
    rank_df.set_index('Node',inplace=True)
    
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 10', b'Error', 0)
    raise ValueError("Error")



In [23]:
#Permanent cell 11
#Run dissolve

try:
    if run_dissolve:
        out_path = global_output_folder + '\\' + gdb_name
        arcpy.env.workspace = out_path  
        arcpy.env.addOutputsToMap = False
        if run_dissolve:
            if arcpy.Exists(gdb_name_dissolve):
                arcpy.management.Delete(gdb_path)
            arcpy.management.CreateFileGDB(global_output_folder, gdb_name_dissolve)
            dissolve_path = global_output_folder + '\\' + gdb_name_dissolve
            arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'Node_Catchment')
        #     arcpy.env.workspace = dissolve_path
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Drains_To", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID", "TEXT")
            arcpy.management.AddField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "TEXT")
            arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID", "!muid!", "PYTHON3")
            for index, row in merge_df.iterrows():
                arcpy.management.CalculateField(dissolve_path + '\\Node_Catchment', "Merge_ID_Temp", "''", "PYTHON3")
                nodes = list(rank_df[rank_df.Merge_ID==row["Merge_ID"]].index)
                print(f'Dissolving for {row["Merge_ID"]}, {index} of {max(merge_df.index)} at time {datetime.datetime.now()}')
                if row['Catchment_Count'] == 1:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    where_clause = f"muid = '{row['To_Dissolve'][0]}'"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.conversion.FeatureClassToFeatureClass('temp_layer', dissolve_path, 'Dissolve_Temp')
                    arcpy.management.Delete("temp_layer")
                    
                else:
                    arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
                    catchments = row['To_Dissolve']
                    catchments_sql = ', '.join([f"'{muid}'" for muid in catchments])
                    where_clause = f"Merge_ID in ({catchments_sql})"
                    arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
                    arcpy.management.CalculateField("temp_layer", "Merge_ID_Temp", f"'{row['Merge_ID']}'", "PYTHON3")
                    arcpy.management.Dissolve("temp_layer",dissolve_path + '\\Dissolve_Temp', "Merge_ID_Temp", "", "MULTI_PART")
                    arcpy.management.Delete("temp_layer")
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Merge_ID", f"'{row['Merge_ID']}'", "PYTHON3")

                for node in nodes:
                    arcpy.management.CalculateField(dissolve_path + '\\Dissolve_Temp', "Drains_To", f"'{node}'", "PYTHON3")
                    arcpy.management.Append(dissolve_path + '\\Dissolve_Temp', dissolve_path + '\\Node_Catchment', "NO_TEST")

                arcpy.management.Delete(dissolve_path + '\\Dissolve_Temp')



        #Delete the features without a Drains_To
        arcpy.management.MakeFeatureLayer(dissolve_path + '\\Node_Catchment', "temp_layer")
        where_clause = f"Drains_To IS NULL"
        arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", where_clause)
        arcpy.management.DeleteFeatures("temp_layer")  
        arcpy.management.Delete("temp_layer")

        #Append the features into the official Node_Catchment layer
        arcpy.management.MakeFeatureLayer("Node_Catchment", "Temp_Layer")
        arcpy.management.DeleteFeatures("Temp_Layer")
        arcpy.management.Append(dissolve_path + '\\Node_Catchment', "Temp_Layer", "NO_TEST")
        arcpy.management.Delete("temp_layer")
        
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 11', b'Error', 0)
    raise ValueError("Error")




Dissolving for Merge_ID_300, 0 of 441 at time 2024-06-05 15:44:28.602808
Dissolving for Merge_ID_373, 1 of 441 at time 2024-06-05 15:45:04.027567
Dissolving for Merge_ID_150, 2 of 441 at time 2024-06-05 15:45:23.638702
Dissolving for Merge_ID_5, 3 of 441 at time 2024-06-05 15:45:51.988420
Dissolving for Merge_ID_201, 4 of 441 at time 2024-06-05 15:46:13.103938
Dissolving for Merge_ID_324, 5 of 441 at time 2024-06-05 15:47:25.930769
Dissolving for Merge_ID_423, 6 of 441 at time 2024-06-05 15:47:47.611312
Dissolving for Merge_ID_183, 7 of 441 at time 2024-06-05 15:48:32.565872
Dissolving for Merge_ID_6, 8 of 441 at time 2024-06-05 15:49:24.860210
Dissolving for Merge_ID_87, 9 of 441 at time 2024-06-05 15:50:19.318056
Dissolving for Merge_ID_291, 10 of 441 at time 2024-06-05 15:51:07.318921
Dissolving for Merge_ID_427, 11 of 441 at time 2024-06-05 15:51:47.375966
Dissolving for Merge_ID_234, 12 of 441 at time 2024-06-05 15:52:11.329112
Dissolving for Merge_ID_50, 13 of 441 at time 2024-06

Dissolving for Merge_ID_161, 111 of 441 at time 2024-06-05 16:36:31.034682
Dissolving for Merge_ID_303, 112 of 441 at time 2024-06-05 16:36:45.814293
Dissolving for Merge_ID_268, 113 of 441 at time 2024-06-05 16:37:09.395009
Dissolving for Merge_ID_61, 114 of 441 at time 2024-06-05 16:37:42.072103
Dissolving for Merge_ID_322, 115 of 441 at time 2024-06-05 16:38:02.270705
Dissolving for Merge_ID_435, 116 of 441 at time 2024-06-05 16:38:44.151274
Dissolving for Merge_ID_386, 117 of 441 at time 2024-06-05 16:39:35.385458
Dissolving for Merge_ID_387, 118 of 441 at time 2024-06-05 16:39:59.398573
Dissolving for Merge_ID_408, 119 of 441 at time 2024-06-05 16:40:14.603576
Dissolving for Merge_ID_397, 120 of 441 at time 2024-06-05 16:40:43.815982
Dissolving for Merge_ID_400, 121 of 441 at time 2024-06-05 16:41:03.399012
Dissolving for Merge_ID_163, 122 of 441 at time 2024-06-05 16:41:45.587853
Dissolving for Merge_ID_217, 123 of 441 at time 2024-06-05 16:42:05.843501
Dissolving for Merge_ID_11

Dissolving for Merge_ID_254, 220 of 441 at time 2024-06-05 17:29:21.380545
Dissolving for Merge_ID_92, 221 of 441 at time 2024-06-05 17:30:12.465474
Dissolving for Merge_ID_177, 222 of 441 at time 2024-06-05 17:31:08.984833
Dissolving for Merge_ID_173, 223 of 441 at time 2024-06-05 17:31:50.872304
Dissolving for Merge_ID_178, 224 of 441 at time 2024-06-05 17:32:42.513734
Dissolving for Merge_ID_24, 225 of 441 at time 2024-06-05 17:33:20.147903
Dissolving for Merge_ID_305, 226 of 441 at time 2024-06-05 17:33:37.915221
Dissolving for Merge_ID_200, 227 of 441 at time 2024-06-05 17:34:42.080153
Dissolving for Merge_ID_147, 228 of 441 at time 2024-06-05 17:35:35.095581
Dissolving for Merge_ID_372, 229 of 441 at time 2024-06-05 17:35:57.056750
Dissolving for Merge_ID_197, 230 of 441 at time 2024-06-05 17:36:24.289756
Dissolving for Merge_ID_417, 231 of 441 at time 2024-06-05 17:36:42.091102
Dissolving for Merge_ID_432, 232 of 441 at time 2024-06-05 17:37:19.445148
Dissolving for Merge_ID_227

Dissolving for Merge_ID_258, 329 of 441 at time 2024-06-05 18:33:07.420442
Dissolving for Merge_ID_418, 330 of 441 at time 2024-06-05 18:33:33.465896
Dissolving for Merge_ID_317, 331 of 441 at time 2024-06-05 18:34:41.888615
Dissolving for Merge_ID_213, 332 of 441 at time 2024-06-05 18:35:01.243357
Dissolving for Merge_ID_193, 333 of 441 at time 2024-06-05 18:35:31.741816
Dissolving for Merge_ID_289, 334 of 441 at time 2024-06-05 18:36:24.526197
Dissolving for Merge_ID_48, 335 of 441 at time 2024-06-05 18:36:43.792856
Dissolving for Merge_ID_437, 336 of 441 at time 2024-06-05 18:37:19.331589
Dissolving for Merge_ID_146, 337 of 441 at time 2024-06-05 18:37:49.594828
Dissolving for Merge_ID_223, 338 of 441 at time 2024-06-05 18:38:19.897603
Dissolving for Merge_ID_82, 339 of 441 at time 2024-06-05 18:39:01.317566
Dissolving for Merge_ID_228, 340 of 441 at time 2024-06-05 18:39:31.790999
Dissolving for Merge_ID_389, 341 of 441 at time 2024-06-05 18:40:45.353422
Dissolving for Merge_ID_235

Dissolving for Merge_ID_170, 438 of 441 at time 2024-06-05 19:47:51.568384
Dissolving for Merge_ID_329, 439 of 441 at time 2024-06-05 19:48:19.767196
Dissolving for Merge_ID_208, 440 of 441 at time 2024-06-05 19:49:00.566541
Dissolving for Merge_ID_357, 441 of 441 at time 2024-06-05 19:49:28.856436


In [24]:
#Permanent cell 12
#Export jpgs
try:
    if run_jpg:
        aprx = arcpy.mp.ArcGISProject("CURRENT")
        project_path = aprx.filePath

        jpg_folder = model_output_folder + r'\jpg'
        if not os.path.isdir(jpg_folder): os.makedirs(jpg_folder) 


        # project_directory = os.path.dirname(project_path)

        layouts = aprx.listLayouts()
        export_fails = []

        for layout in layouts:
            if layout.mapSeries is not None:
                map_series = layout.mapSeries
                map_series.refresh()
                # Loop through all pages in the map series
                for page_number in range(1, map_series.pageCount + 1):
                    map_series.currentPageNumber = page_number
                    output_filename = os.path.join(jpg_folder, f"{map_series.pageRow.Drains_To}.jpg")
                    try:
                        layout.exportToJPEG(output_filename, resolution=300)
                    except:
                        print(f'WARNING! {map_series.pageRow.Drains_To} could not be made')
                        export_fails.append(map_series.pageRow.Drains_To)
                    print (f'Printing jpg {page_number} of {map_series.pageCount} at time {datetime.datetime.now()}')

        print(f'the following pages failed: {export_fails}')            
        print("Export complete.")

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 12', b'Error', 0)
    raise ValueError("Error")



Printing jpg 1 of 1755 at time 2024-06-05 19:50:34.866859
Printing jpg 2 of 1755 at time 2024-06-05 19:50:40.644147
Printing jpg 3 of 1755 at time 2024-06-05 19:50:46.286312
Printing jpg 4 of 1755 at time 2024-06-05 19:50:51.923471
Printing jpg 5 of 1755 at time 2024-06-05 19:50:57.321412
Printing jpg 6 of 1755 at time 2024-06-05 19:51:04.462948
Printing jpg 7 of 1755 at time 2024-06-05 19:51:12.640432
Printing jpg 8 of 1755 at time 2024-06-05 19:51:20.430562
Printing jpg 9 of 1755 at time 2024-06-05 19:51:28.055545
Printing jpg 10 of 1755 at time 2024-06-05 19:51:31.873035
Printing jpg 11 of 1755 at time 2024-06-05 19:51:35.532384
Printing jpg 12 of 1755 at time 2024-06-05 19:51:39.354883
Printing jpg 13 of 1755 at time 2024-06-05 19:51:42.318595
Printing jpg 14 of 1755 at time 2024-06-05 19:51:46.235180
Printing jpg 15 of 1755 at time 2024-06-05 19:51:50.241846
Printing jpg 16 of 1755 at time 2024-06-05 19:51:54.112390
Printing jpg 17 of 1755 at time 2024-06-05 19:51:58.727614
Printi

Printing jpg 139 of 1755 at time 2024-06-05 20:04:51.731637
Printing jpg 140 of 1755 at time 2024-06-05 20:04:57.045499
Printing jpg 141 of 1755 at time 2024-06-05 20:05:01.087199
Printing jpg 142 of 1755 at time 2024-06-05 20:05:04.943728
Printing jpg 143 of 1755 at time 2024-06-05 20:05:08.553031
Printing jpg 144 of 1755 at time 2024-06-05 20:05:12.635767
Printing jpg 145 of 1755 at time 2024-06-05 20:05:16.312131
Printing jpg 146 of 1755 at time 2024-06-05 20:05:20.108606
Printing jpg 147 of 1755 at time 2024-06-05 20:05:24.237384
Printing jpg 148 of 1755 at time 2024-06-05 20:05:27.644502
Printing jpg 149 of 1755 at time 2024-06-05 20:05:31.183741
Printing jpg 150 of 1755 at time 2024-06-05 20:05:36.058202
Printing jpg 151 of 1755 at time 2024-06-05 20:05:40.750496
Printing jpg 152 of 1755 at time 2024-06-05 20:05:44.943333
Printing jpg 153 of 1755 at time 2024-06-05 20:05:49.563562
Printing jpg 154 of 1755 at time 2024-06-05 20:05:53.222910
Printing jpg 155 of 1755 at time 2024-06

Printing jpg 275 of 1755 at time 2024-06-05 20:21:56.293477
Printing jpg 276 of 1755 at time 2024-06-05 20:21:59.707601
Printing jpg 277 of 1755 at time 2024-06-05 20:22:02.711349
Printing jpg 278 of 1755 at time 2024-06-05 20:22:05.803178
Printing jpg 279 of 1755 at time 2024-06-05 20:22:09.748788
Printing jpg 280 of 1755 at time 2024-06-05 20:22:13.100855
Printing jpg 281 of 1755 at time 2024-06-05 20:22:16.026532
Printing jpg 282 of 1755 at time 2024-06-05 20:22:21.615646
Printing jpg 283 of 1755 at time 2024-06-05 20:22:26.459077
Printing jpg 284 of 1755 at time 2024-06-05 20:22:31.786334
Printing jpg 285 of 1755 at time 2024-06-05 20:22:37.232317
Printing jpg 286 of 1755 at time 2024-06-05 20:22:42.862469
Printing jpg 287 of 1755 at time 2024-06-05 20:22:48.610728
Printing jpg 288 of 1755 at time 2024-06-05 20:22:52.167982
Printing jpg 289 of 1755 at time 2024-06-05 20:22:56.047532
Printing jpg 290 of 1755 at time 2024-06-05 20:22:59.742913
Printing jpg 291 of 1755 at time 2024-06

Printing jpg 411 of 1755 at time 2024-06-05 20:37:48.940782
Printing jpg 412 of 1755 at time 2024-06-05 20:37:52.926427
Printing jpg 413 of 1755 at time 2024-06-05 20:37:57.868949
Printing jpg 414 of 1755 at time 2024-06-05 20:38:02.299001
Printing jpg 415 of 1755 at time 2024-06-05 20:38:08.447625
Printing jpg 416 of 1755 at time 2024-06-05 20:38:13.013802
Printing jpg 417 of 1755 at time 2024-06-05 20:38:17.711099
Printing jpg 418 of 1755 at time 2024-06-05 20:38:27.110698
Printing jpg 419 of 1755 at time 2024-06-05 20:38:36.339139
Printing jpg 420 of 1755 at time 2024-06-05 20:38:42.325615
Printing jpg 421 of 1755 at time 2024-06-05 20:38:46.303254
Printing jpg 422 of 1755 at time 2024-06-05 20:38:50.159782
Printing jpg 423 of 1755 at time 2024-06-05 20:38:55.691842
Printing jpg 424 of 1755 at time 2024-06-05 20:39:07.168340
Printing jpg 425 of 1755 at time 2024-06-05 20:39:14.869385
Printing jpg 426 of 1755 at time 2024-06-05 20:39:21.542489
Printing jpg 427 of 1755 at time 2024-06

Printing jpg 547 of 1755 at time 2024-06-05 20:53:56.611934
Printing jpg 548 of 1755 at time 2024-06-05 20:54:00.865825
Printing jpg 549 of 1755 at time 2024-06-05 20:54:03.675395
Printing jpg 550 of 1755 at time 2024-06-05 20:54:06.815266
Printing jpg 551 of 1755 at time 2024-06-05 20:54:09.505727
Printing jpg 552 of 1755 at time 2024-06-05 20:54:12.662615
Printing jpg 553 of 1755 at time 2024-06-05 20:54:16.227876
Printing jpg 554 of 1755 at time 2024-06-05 20:54:19.528895
Printing jpg 555 of 1755 at time 2024-06-05 20:54:22.618721
Printing jpg 556 of 1755 at time 2024-06-05 20:54:25.726563
Printing jpg 557 of 1755 at time 2024-06-05 20:54:29.066619
Printing jpg 558 of 1755 at time 2024-06-05 20:54:32.183469
Printing jpg 559 of 1755 at time 2024-06-05 20:54:35.556555
Printing jpg 560 of 1755 at time 2024-06-05 20:54:38.968676
Printing jpg 561 of 1755 at time 2024-06-05 20:54:42.101541
Printing jpg 562 of 1755 at time 2024-06-05 20:54:45.332496
Printing jpg 563 of 1755 at time 2024-06

...

Printing jpg 1090 of 1755 at time 2024-06-05 21:33:58.240575
Printing jpg 1091 of 1755 at time 2024-06-05 21:34:02.620581
Printing jpg 1092 of 1755 at time 2024-06-05 21:34:07.422971
Printing jpg 1093 of 1755 at time 2024-06-05 21:34:11.837008
Printing jpg 1094 of 1755 at time 2024-06-05 21:34:16.682438
Printing jpg 1095 of 1755 at time 2024-06-05 21:34:22.039376
Printing jpg 1096 of 1755 at time 2024-06-05 21:34:26.652594
Printing jpg 1097 of 1755 at time 2024-06-05 21:34:32.273734
Printing jpg 1098 of 1755 at time 2024-06-05 21:34:38.515441
Printing jpg 1099 of 1755 at time 2024-06-05 21:34:44.264698
Printing jpg 1100 of 1755 at time 2024-06-05 21:34:48.742792
Printing jpg 1101 of 1755 at time 2024-06-05 21:34:54.398964
Printing jpg 1102 of 1755 at time 2024-06-05 21:34:59.436570
Printing jpg 1103 of 1755 at time 2024-06-05 21:35:03.913664
Printing jpg 1104 of 1755 at time 2024-06-05 21:35:06.888384
Printing jpg 1105 of 1755 at time 2024-06-05 21:35:10.111991
Printing jpg 1106 of 175

Printing jpg 1224 of 1755 at time 2024-06-05 21:48:54.004697
Printing jpg 1225 of 1755 at time 2024-06-05 21:49:04.554342
Printing jpg 1226 of 1755 at time 2024-06-05 21:49:15.022913
Printing jpg 1227 of 1755 at time 2024-06-05 21:49:25.608094
Printing jpg 1228 of 1755 at time 2024-06-05 21:49:36.074663
Printing jpg 1229 of 1755 at time 2024-06-05 21:49:49.575005
Printing jpg 1230 of 1755 at time 2024-06-05 21:50:00.393896
Printing jpg 1231 of 1755 at time 2024-06-05 21:50:12.624078
Printing jpg 1232 of 1755 at time 2024-06-05 21:50:23.105661
Printing jpg 1233 of 1755 at time 2024-06-05 21:50:33.500164
Printing jpg 1234 of 1755 at time 2024-06-05 21:50:38.066338
Printing jpg 1235 of 1755 at time 2024-06-05 21:50:42.891750
Printing jpg 1236 of 1755 at time 2024-06-05 21:50:53.103086
Printing jpg 1237 of 1755 at time 2024-06-05 21:51:03.400500
Printing jpg 1238 of 1755 at time 2024-06-05 21:51:14.031219
Printing jpg 1239 of 1755 at time 2024-06-05 21:51:24.403205
Printing jpg 1240 of 175

Printing jpg 1358 of 1755 at time 2024-06-05 22:10:18.266382
Printing jpg 1359 of 1755 at time 2024-06-05 22:10:24.254856
Printing jpg 1360 of 1755 at time 2024-06-05 22:10:30.250337
Printing jpg 1361 of 1755 at time 2024-06-05 22:10:36.070658
Printing jpg 1362 of 1755 at time 2024-06-05 22:10:41.096252
Printing jpg 1363 of 1755 at time 2024-06-05 22:10:45.990726
Printing jpg 1364 of 1755 at time 2024-06-05 22:10:50.587928
Printing jpg 1365 of 1755 at time 2024-06-05 22:10:55.769665
Printing jpg 1366 of 1755 at time 2024-06-05 22:11:00.537023
Printing jpg 1367 of 1755 at time 2024-06-05 22:11:06.857801
Printing jpg 1368 of 1755 at time 2024-06-05 22:11:13.280673
Printing jpg 1369 of 1755 at time 2024-06-05 22:11:19.654499
Printing jpg 1370 of 1755 at time 2024-06-05 22:11:24.687100
Printing jpg 1371 of 1755 at time 2024-06-05 22:11:29.785761
Printing jpg 1372 of 1755 at time 2024-06-05 22:11:33.979099
Printing jpg 1373 of 1755 at time 2024-06-05 22:11:38.306054
Printing jpg 1374 of 175

Printing jpg 1492 of 1755 at time 2024-06-05 22:19:17.723078
Printing jpg 1493 of 1755 at time 2024-06-05 22:19:21.566591
Printing jpg 1494 of 1755 at time 2024-06-05 22:19:25.533218
Printing jpg 1495 of 1755 at time 2024-06-05 22:19:29.347705
Printing jpg 1496 of 1755 at time 2024-06-05 22:19:33.522521
Printing jpg 1497 of 1755 at time 2024-06-05 22:19:37.656825
Printing jpg 1498 of 1755 at time 2024-06-05 22:19:41.662486
Printing jpg 1499 of 1755 at time 2024-06-05 22:19:45.403907
Printing jpg 1500 of 1755 at time 2024-06-05 22:19:49.245418
Printing jpg 1501 of 1755 at time 2024-06-05 22:19:53.251080
Printing jpg 1502 of 1755 at time 2024-06-05 22:19:57.102601
Printing jpg 1503 of 1755 at time 2024-06-05 22:20:01.132285
Printing jpg 1504 of 1755 at time 2024-06-05 22:20:04.962786
Printing jpg 1505 of 1755 at time 2024-06-05 22:20:09.104572
Printing jpg 1506 of 1755 at time 2024-06-05 22:20:12.960097
Printing jpg 1507 of 1755 at time 2024-06-05 22:20:16.915713
Printing jpg 1508 of 175

Printing jpg 1626 of 1755 at time 2024-06-05 22:29:08.394537
Printing jpg 1627 of 1755 at time 2024-06-05 22:29:13.499203
Printing jpg 1628 of 1755 at time 2024-06-05 22:29:18.789038
Printing jpg 1629 of 1755 at time 2024-06-05 22:29:24.064860
Printing jpg 1630 of 1755 at time 2024-06-05 22:29:29.355697
Printing jpg 1631 of 1755 at time 2024-06-05 22:29:34.860728
Printing jpg 1632 of 1755 at time 2024-06-05 22:29:40.963307
Printing jpg 1633 of 1755 at time 2024-06-05 22:29:46.687042
Printing jpg 1634 of 1755 at time 2024-06-05 22:29:52.631475
Printing jpg 1635 of 1755 at time 2024-06-05 22:29:58.364716
Printing jpg 1636 of 1755 at time 2024-06-05 22:30:03.719611
Printing jpg 1637 of 1755 at time 2024-06-05 22:30:09.019455
Printing jpg 1638 of 1755 at time 2024-06-05 22:30:13.950962
Printing jpg 1639 of 1755 at time 2024-06-05 22:30:19.531063
Printing jpg 1640 of 1755 at time 2024-06-05 22:30:24.831908
Printing jpg 1641 of 1755 at time 2024-06-05 22:30:29.989623
Printing jpg 1642 of 175

In [152]:
#Permanent cell 13
#Create spreadsheets

try:
    hex_blue = "ADD8E6"
    hex_yellow = "FFFACD"
    border_style = Side(style='thin', color='000000')
    border = Border(top=border_style, bottom=border_style, left=border_style, right=border_style)
    border_style_none = Side(style=None, color='000000')
    border_none = Border(top=border_style_none, bottom=border_style_none, left=border_style_none, right=border_style_none)

    excel_folder = model_output_folder + '\\Excel'
    img_folder = model_output_folder + '\\jpg'
    if not os.path.isdir(excel_folder): os.makedirs(excel_folder) 
    if not os.path.isdir(img_folder): os.makedirs(img_folder) 
    for id in node_df[('GENERAL INFO','ID')].unique():    
        node_single_df = node_df[node_df[('GENERAL INFO','ID')]==id].copy()
        node_single_df.reset_index(drop=True,inplace=True)
        
        
        if use_formula: #Replace spreadsheet values with formulas
            value_startrow = 17
            for i in node_single_df.index:
                currentrow = i + value_startrow
                for avg_calc_dict_key in avg_calc_dict:
#                     currentrow = i + value_startrow
                    inputs = avg_calc_dict[avg_calc_dict_key]
                    header = inputs[0]
                    subheader = inputs[2]
                    unitflow_ref = inputs[3]
                    col_ref = inputs[4]
                    formula = f'{r"="}{unitflow_ref}*{col_ref}{currentrow}/86400'
                    node_single_df.loc[i,(header,subheader)] = formula                
                
                node_single_df.loc[i,('RESIDENTIAL','PEAK FLOW (L/s)')] = f'{r"="}MAX(1.5,(1+(14/(4+((H{currentrow}/1000)^0.5)))))*I{currentrow}'
                node_single_df.loc[i,('COMMERCIAL','PEAK FLOW (L/s)')] = \
                    f'{r"="}MAX(1.5,(1+(14/(4+((({avg_calc_dict["com"][3]}*K{currentrow})/({avg_calc_dict["res"][3]}*1000))^0.5))))*0.8)*L{currentrow}'
                
                ind_formula = f'{r"="}MAX(1.5,0.8*(1+((14)/(4+(((N{currentrow}*{avg_calc_dict["ind"][3]}/{avg_calc_dict["res"][3]}'
                ind_formula += f')/1000)^0.5))))*IF(N{currentrow}<121,1.7,(2.505-0.1673*LN(N{currentrow}))))*O{currentrow}'
                node_single_df.loc[i,('INDUSTRIAL','PEAK FLOW (L/s)')] = ind_formula
                
                node_single_df.loc[i,('INSTITUTIONAL','PEAK FLOW (L/s)')] = \
                    f'{r"="}MAX(1.5,(1+(14/(4+((({avg_calc_dict["inst"][3]}*Q{currentrow})/({avg_calc_dict["res"][3]}*1000))^0.5)))))*R{currentrow}'

                node_single_df.loc[i,('INFLOW / INFILTRATION','AREA (Ha)')] = f'{r"="}G{currentrow}+K{currentrow}+N{currentrow}+Q{currentrow}'
                node_single_df.loc[i,('INFLOW / INFILTRATION','INFLOW (L/s)')] = f'{r"="}T{currentrow}*{avg_calc_dict["infl"][3]}/86400'
                node_single_df.loc[i,('INFLOW / INFILTRATION','INFILTRATION (L/s)')] = f'{r"="}T{currentrow}*{avg_calc_dict["infi"][3]}/86400'
                
                node_single_df.loc[i,('FLOWS','AVG. SAN. FLOW (L/s)')] = f'{r"="}I{currentrow}+L{currentrow}+O{currentrow}+R{currentrow}'
                node_single_df.loc[i,('FLOWS','ADWF (L/s)')] = f'{r"="}W{currentrow}+V{currentrow}'
                node_single_df.loc[i,('FLOWS','PWWF (L/s)')] = \
                    f'{r"="}J{currentrow}+M{currentrow}+P{currentrow}+S{currentrow}+U{currentrow}+V{currentrow}'
                
        id = id if not '/' in id else id.replace('/','_')
        id = id.replace('\\','_')
        muid = node_single_df.iloc[0,0]

        sheetpath = excel_folder + "\\" + id + ".xlsx"
        startrow = 13
        with pd.ExcelWriter(sheetpath) as writer:
            node_single_df.to_excel(writer, sheet_name=id,startrow=startrow)
            info_df.to_excel(writer, sheet_name=id,startrow=1,startcol=2)

            workbook = writer.book
            workbook.create_sheet("Map")

        workbook = load_workbook(sheetpath)    
        sheet1 = workbook[id]

        #Format infobox
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=3,max_col=5):
            for cell in col[1:2]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[2:7]:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = PatternFill(start_color=hex_yellow, end_color=hex_yellow, fill_type="solid")
                cell.border = border
        sheet1.column_dimensions['C'].width = 22 
        sheet1.column_dimensions['D'].width = 11 

        #Remove index
        for row in sheet1.iter_rows():
            for cell in row[:1]:
                cell.value = ''
                cell.border = border_none
        #Format main table header rows
        merged_range = sheet1.merged_cells
        for col in sheet1.iter_cols(min_col=2):
            for cell in col[startrow+1:startrow+2]:
                cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
                cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")
            for cell in col[startrow:startrow+1]:
                if cell.coordinate in sheet1.merged_cells:
                    cell.fill = PatternFill(start_color=hex_blue, end_color=hex_blue, fill_type="solid")        

        sheet1.column_dimensions['C'].width = 22 
        sheet1.column_dimensions['D'].width = 11   
        sheet1.column_dimensions['F'].width = 13
        sheet1.column_dimensions['H'].width = 13  
        sheet1.column_dimensions['V'].width = 13     

        sheet = workbook["Map"]

        # Add an image to the sheet
        img_path = img_folder + '\\' + muid + '.jpg'  # Replace with the path to your image
        img = Image(img_path)

        # Set the position for the image (e.g., 'B2' for cell B2)
        sheet.add_image(img, 'B2')

        workbook.save(sheetpath)


except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 13', b'Error', 0)
    raise ValueError("Error")



In [150]:
for i in node_single_df.index:
    print(i)

0
1
2


In [151]:
i

2

In [104]:
#Permanent cell 14
#Create HTMLs

try:
    if run_html:
        html_folder = model_output_folder + '\\HTML'
        
        shutil.copy2('style.css', html_folder + '\\style.css')
        shutil.copy2('script.js', html_folder + '\\script.js')

        for category in categories:
            area_type = category[0]
            area_names = category[1]
            header_start = category[2]

            f = open(html_folder + '\\Population_By_' + area_type + '_' + model_area + '.html', "w")
            f.write('<link rel="stylesheet" href="style.css">\n')
            f.write('<script src="script.js"></script>\n')
            f.write('<link rel="stylesheet" href="style.css">\n')
            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<meta charset="utf-8">\n')
            f.write('</head>\n')
            f.write('<body>\n\n')

            f.write('<div class="tab">\n')
            for area_name in area_names:
                tab = area_name

            #     color = ps_dict[first_year]
            #     bg_color = color_dict[color][0]
            #     text_color = color_dict[color][1]

                f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
            f.write('</div>\n')

            pop_df = pop_dfss[0][2]

            for area_name in area_names:

                area_df = pop_df[pop_df[area_type]==area_name]
                area_df = area_df[['Year','Population']].groupby(['Year']).sum()

                f.write('<div id="' + area_name + '" class="tabcontent">\n') 
                f.write('<h1>' + area_name + '</h1>\n')

                f.write('<div class="sidenav">\n')

                f.write('<table style=\'width: 90%;\'>\n')
                f.write('<tr>\n')
                f.write('<th>Year</th>\n')
                f.write('<th>Population</th>\n')
                f.write('</tr>\n')

                for index, row in area_df.iterrows():
                    f.write('<tr>\n')
                    f.write('<td>'+ str(index) + '</td>\n')
                    population_with_separator = f"{int(row['Population']):,}"
                    f.write('<td>'+ population_with_separator + '</td>\n')

                    f.write('</tr>\n')
                f.write('</table>\n')

                for i in range(4):
                    f.write('<h1 style="color: white">End of tables</h1>\n')#Invisible, just to enable scroll to table bottoms

                f.write('</div>\n') #end sidenav


                f.write('<div class="main">\n')

                fig = go.Figure()


                fig.add_trace(go.Scatter(x=area_df.index, 
                                             y = area_df.Population, 
                                             mode='lines',name=pop_dfss[0][0],line=dict(width=5)))

                for pop_dfs in pop_dfss[1:]:
                    pop_df_past = pop_dfs[2]
                    area_df = pop_df_past[pop_df_past[area_type]==area_name]
                    area_df = area_df[['Year','Population']].groupby(['Year']).sum()
                    fig.add_trace(go.Scatter(x=area_df.index, 
                                             y = area_df.Population, 
                                             mode='lines',name=pop_dfs[0],line=dict(width=2)))

                fig.update_layout(
                    title=header_start + area_name,
                    autosize=False,
                    width = 1500,
                    height=850,
                    margin=dict(
                        l=50,
                        r=50,
                        b=50,
                        t=50,
                        pad=4
                        ),
                        yaxis_title = 'Population'
                    )

                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

                f.write('</div>\n') #end div main  

                f.write('</div>\n')  #end div tab   

                f.write('</body>\n')
            f.write('</html>\n')
            f.close()
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 14', b'Error', 0)
    raise ValueError("Error")



In [ ]:
#Obsolete cell
# #Dissolve catchments
# arcpy.env.addOutputsToMap = False
# if run_dissolve:
#     arcpy.management.CreateFileGDB(output_folder, gdb_name_dissolve)
#     dissolve_path = output_folder + '\\' + gdb_name_dissolve
#     arcpy.conversion.FeatureClassToFeatureClass('msm_Catchment', dissolve_path, 'msm_Catchment')
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):
#         print('Dissolving for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         catchment_df = accumulation_df[accumulation_df[('GENERAL INFO','NODE')]==node]
#         catchments = list(catchment_df[('GENERAL INFO','CATCHMENT')].unique())
#         arcpy.management.CalculateField(dissolve_path + '\\msm_Catchment', "Drains_To", "''", "PYTHON3")
#         with arcpy.da.UpdateCursor(dissolve_path + '\\msm_catchment', ['muid', 'Drains_To']) as cursor:
#             for row in cursor:
#                 if row[0] in catchments:
#                     row[1] = node
#                     cursor.updateRow(row)

#         query = "Drains_To = 'Test'"
#         arcpy.management.MakeFeatureLayer(dissolve_path + '\\msm_catchment', "temp_layer", "Drains_To = '" + node + "'")
#         dissolve_output = dissolve_path + '\\msm_Catchment_Dissolve_Single'
#         arcpy.management.Dissolve("temp_layer", dissolve_output, "Drains_To", "", "MULTI_PART")
#         arcpy.management.Delete("temp_layer")

#         arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\msm_Catchment_Dissolve_Single', dissolve_path, 'Node_Catchment_' + node)



In [ ]:
#Obsolete cell
#Append individual dissolved catchments to one layer.

# if run_dissolve_append:
#     nodes = list(accumulation_df[('GENERAL INFO','NODE')].unique())
#     for i, node in enumerate(nodes):    
#         print('Appending for node ' + str(i) + ' of ' + str(len(nodes)) + ' at time ' + str(datetime.datetime.now()))
#         if i == 0:
#             arcpy.conversion.FeatureClassToFeatureClass(dissolve_path + '\\Node_Catchment_' + node, out_path, 'Node_Catchment')
#         else:
#             arcpy.management.Append(dissolve_path + '\\Node_Catchment_' + node, "Node_Catchment", "NO_TEST")
